# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [36]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})
print(movie_item)
for p in movie_item:
    print(p)
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

[<option data-name="復仇母親" value="10392">復仇母親</option>, <option data-name="風暴過後" value="10387">風暴過後</option>, <option data-name="南方車站的聚會" value="10380">南方車站的聚會</option>, <option data-name="祭旗" value="10373">祭旗</option>, <option data-name="唯你是愛" value="10370">唯你是愛</option>, <option data-name="屍控療法" value="10367">屍控療法</option>, <option data-name="黑熊來了" value="10357">黑熊來了</option>, <option data-name="唐人街探案2" value="10353">唐人街探案2</option>, <option data-name="黑計畫" value="10351">黑計畫</option>, <option data-name="HUMAN LOST人間失格" value="10348">HUMAN LOST人間失格</option>, <option data-name="假面騎士劇場版ZI-O Over Quartzer" value="10331">假面騎士劇場版ZI-O Over Quartzer</option>, <option data-name="巴黎小情聖" value="10329">巴黎小情聖</option>, <option data-name="每個月來訪一次的月經醬" value="10324">每個月來訪一次的月經醬</option>, <option data-name="8級警戒" value="10323">8級警戒</option>, <option data-name="為你存在的每一天" value="10297">為你存在的每一天</option>, <option data-name="黑色聖誕節" value="10291">黑色聖誕節</option>, <option data-name="熱氣球飛行家" value="10257">熱氣

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [8]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10392

In [40]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [10]:
# 指定放映地區
area_id = 28

In [42]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 16
十二月 17
十二月 18
十二月 19
十二月 20


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [48]:
# 選定要觀看的日期
date = "2019-12-16"

In [66]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})
print(html)

[<ul class="area_time _c jq_area_time" data-theater_name="台北in89豪華數位影城" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=36" data-theater_url="http://www.in89.com.tw/website_module.php?website_module_classify_sn=39" id="theater_id_36">
<li class="adds">
<a href="https://movies.yahoo.com.tw/theater_result.html/id=36">台北in89豪華數位影城</a>
<span>02-23315077</span>
</li>
<li class="taps">
<span class="tapR">數位</span> </li>
<li class="time _c">
<div class="input_picker jq_input_picker">
<input class="gabtn" data-cinema="e9c365f27f4e11e494ae8b4ddf78b93a" data-date="1576425600" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '復仇母親']" data-movie="e6c496a3aaf5458d965d4639a6479293" data-movie_date="12.16" data-movie_time="17:50" data-movie_title="復仇母親" data-movie_type="數位" data-session="60fc961013234bd28afe4b720d13ae6d" data-time="1576489800" id="72589971" name="schedule_list" type="radio" value="2019-12-16 17:50:00"/>
<label class="select" for="72589971">17:50</lab

In [118]:

for theater in html:
    print("電影院："+ theater['data-theater_name'])
    print("放映類型："+theater.find(class_='tapR').string)
    
    for gabtndata in theater.find_all(class_='gabtn'):
        print("放映時間："+gabtndata['value'])
#        print("放映類型："+gabtndata['data-movie_type']," 時間："+gabtndata['data-movie_time'])
    print('-------------------------------')
    

電影院：台北in89豪華數位影城
放映類型：數位
放映時間：2019-12-16 17:50:00
放映時間：2019-12-16 22:05:00
-------------------------------
電影院：喜滿客絕色影城
放映類型：數位
放映時間：2019-12-16 12:25:00
放映時間：2019-12-16 16:25:00
放映時間：2019-12-16 20:40:00
-------------------------------
電影院：台北信義威秀影城
放映類型：數位
放映時間：2019-12-16 13:15:00
放映時間：2019-12-16 19:15:00
放映時間：2019-12-16 23:55:00
-------------------------------
電影院：哈拉影城
放映類型：數位
放映時間：2019-12-16 13:10:00
放映時間：2019-12-16 18:10:00
放映時間：2019-12-16 22:10:00
-------------------------------
電影院：百老匯數位影城
放映類型：數位
放映時間：2019-12-16 09:50:00
放映時間：2019-12-16 13:55:00
放映時間：2019-12-16 18:00:00
放映時間：2019-12-16 22:05:00
放映時間：2019-12-17 00:00:00
-------------------------------
電影院：今日秀泰影城
放映類型：數位
放映時間：2019-12-16 10:20:00
放映時間：2019-12-16 14:50:00
放映時間：2019-12-16 21:40:00
-------------------------------
電影院：喜樂時代影城南港店
放映類型：數位
放映時間：2019-12-16 12:45:00
放映時間：2019-12-16 16:35:00
放映時間：2019-12-16 22:15:00
-------------------------------


In [16]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''


----------------------------------------------------------------------
電影院:  台北美麗華大直影城
放映類型:  數位
2019-08-21 09:00:00
2019-08-21 11:10:00
2019-08-21 13:15:00
2019-08-21 15:20:00
2019-08-21 19:30:00
2019-08-21 21:40:00
2019-08-21 22:30:00
----------------------------------------------------------------------
電影院:  台北新光影城
放映類型:  數位
2019-08-21 10:00:00
2019-08-21 14:50:00
2019-08-21 19:30:00
----------------------------------------------------------------------
電影院:  台北in89豪華數位影城
放映類型:  數位
2019-08-21 09:30:00
2019-08-21 11:20:00
2019-08-21 13:15:00
2019-08-21 15:10:00
2019-08-21 16:10:00
2019-08-21 17:10:00
2019-08-21 18:05:00
2019-08-21 19:10:00
2019-08-21 21:10:00
2019-08-21 23:10:00
2019-08-22 01:10:00
----------------------------------------------------------------------
電影院:  台北日新威秀影城
放映類型:  數位
2019-08-21 09:00:00
2019-08-21 10:55:00
2019-08-21 12:50:00
2019-08-21 14:45:00
2019-08-21 16:40:00
2019-08-21 18:35:00
2019-08-21 20:35:00
-----------------------------------------------------